In [ ]:
import h5py    
import numpy as np  
import pandas as pd
from pylab import savefig
from sklearn.ensemble import IsolationForest
from pylab import *
import pandas as pd
import numpy as np

# ACP
import sklearn.decomposition as sd
import sklearn.preprocessing as sp

# Hierarchical clustering
import scipy.cluster.hierarchy as sch
# LOF
import sklearn.neighbors as sn
# Isolation Forest
import sklearn.ensemble as se




from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()
sb.set_style("whitegrid")


# Data

In [ ]:
f= pd.read_hdf('/home/joevin/Bureau/train.hdf5')

In [ ]:
f=pd.DataFrame(f)

In [ ]:

t=array(f.iloc[4])
temps=np.linspace(0,61440/1000,61440)
plot(temps,t)

In [ ]:
import scipy.stats as stats

# Fourier

In [ ]:
fourier=np.fft.fft(f)

In [ ]:
features= np.zeros((1677,5))
for i in range(1677) : 
    features[i,0]=np.mean(fourier[i])
    features[i,1] = np.var(fourier[i])
    features[i,2] = np.median(fourier[i])
    features[i,3] = np.real(stats.kurtosis(fourier[i],bias=False))
    features[i,4] = np.real(stats.skew(fourier[i],bias=False))

# Wavelet

In [ ]:
import pywt
from pywt import wavedec

from statsmodels.robust import mad

In [ ]:
wf = "haar"
f=np.array(f)
Coeff = []
TCoeff = []
for i in range(1677) :
    #Apply wavelet decomposition
    x=f[i]
    coeffs = pywt.wavedec(x,wf,level=7)
    coeffs_flatten = np.hstack(coeffs)
    Coeff.append(coeffs_flatten)
    
    # Compute universal Threshold http://jseabold.net/blog/2012/02/23/wavelet-regression-in-python/
    sigma = mad(coeffs[-1])
    uthresh = sigma*np.sqrt(2*np.log(128))
    # Apply Threshold on 4 last levels
    coeffs_thresh = [pywt.threshold(c, uthresh, mode="hard") if i<=3 else c for i,c in enumerate(coeffs[::-1])]
    coeffs_thresh_flatten = np.hstack(coeffs_thresh[::-1])
    TCoeff.append(coeffs_thresh_flatten)
    
Coeff = np.array(Coeff)
TCoeff = np.array(TCoeff)
print(Coeff.shape, TCoeff.shape)
print(np.sum(Coeff!=0), np.sum(TCoeff!=0))

In [ ]:
featuresw= np.zeros((1677,10))
for i in range(1677) : 
    featuresw[i,0] = np.mean(Coeff[i])
    featuresw[i,1] = np.var(Coeff[i])
    featuresw[i,2] = np.median(Coeff[i])
    featuresw[i,3] = np.real(stats.kurtosis(Coeff[i],bias=False))
    featuresw[i,4] = np.real(stats.skew(Coeff[i],bias=False))
    featuresw[i,5] = np.mean(TCoeff[i])
    featuresw[i,6] = np.var(TCoeff[i])
    featuresw[i,7] = np.median(TCoeff[i])
    featuresw[i,8] = np.real(stats.kurtosis(TCoeff[i],bias=False))
    featuresw[i,9] = np.real(stats.skew(TCoeff[i],bias=False))

# ACP

In [ ]:
featuresw= np.zeros((1677,10))
features= np.zeros((1677,5))

In [ ]:
feat=np.zeros((1677,15))
feat[:,0:5]=features
feat[:,5:15]=featuresw

In [ ]:
acp = sd.PCA()
acp_features = acp.fit_transform(sp.scale(feat))

# One class svm

In [ ]:
Y_label=np.zeros(1677)

In [ ]:
import sklearn.svm as ssvm
OCS = ssvm.OneClassSVM(kernel="rbf", nu=0.05)

OCS.fit(acp_features)
pred = OCS.predict(acp_features)

CT_svm = pd.DataFrame(list(zip(pred,Y_label)), columns=["pred","Anomaly"])
display(pd.crosstab(CT_svm.pred, CT_svm.Anomaly))

# Test sur validation

In [ ]:
v=pd.read_hdf('/home/joevin/Bureau/validation.hdf5')
v=pd.DataFrame(v)
fourier=np.fft.fft(v)
features= np.zeros((1677,5))
for i in range(1677) : 
    features[i,0]=np.mean(fourier[i])
    features[i,1] = np.var(fourier[i])
    features[i,2] = np.median(fourier[i])
    features[i,3] = np.real(stats.kurtosis(fourier[i],bias=False))
    features[i,4] = np.real(stats.skew(fourier[i],bias=False))
wf = "haar"
v=np.array(v)
Coeff = []
TCoeff = []
for i in range(1677) :
    #Apply wavelet decomposition
    x=v[i]
    coeffs = pywt.wavedec(x,wf,level=7)
    coeffs_flatten = np.hstack(coeffs)
    Coeff.append(coeffs_flatten)
    
    # Compute universal Threshold http://jseabold.net/blog/2012/02/23/wavelet-regression-in-python/
    sigma = mad(coeffs[-1])
    uthresh = sigma*np.sqrt(2*np.log(128))
    # Apply Threshold on 4 last levels
    coeffs_thresh = [pywt.threshold(c, uthresh, mode="hard") if i<=3 else c for i,c in enumerate(coeffs[::-1])]
    coeffs_thresh_flatten = np.hstack(coeffs_thresh[::-1])
    TCoeff.append(coeffs_thresh_flatten)
    
Coeff = np.array(Coeff)
TCoeff = np.array(TCoeff)
print(Coeff.shape, TCoeff.shape)
print(np.sum(Coeff!=0), np.sum(TCoeff!=0))

featuresw= np.zeros((1677,10))
for i in range(1677) : 
    featuresw[i,0] = np.mean(Coeff[i])
    featuresw[i,1] = np.var(Coeff[i])
    featuresw[i,2] = np.median(Coeff[i])
    featuresw[i,3] = np.real(stats.kurtosis(Coeff[i],bias=False))
    featuresw[i,4] = np.real(stats.skew(Coeff[i],bias=False))
    featuresw[i,5] = np.mean(TCoeff[i])
    featuresw[i,6] = np.var(TCoeff[i])
    featuresw[i,7] = np.median(TCoeff[i])
    featuresw[i,8] = np.real(stats.kurtosis(TCoeff[i],bias=False))
    featuresw[i,9] = np.real(stats.skew(TCoeff[i],bias=False))
    
feat=np.zeros((1677,15))
feat[:,0:5]=features
feat[:,5:15]=featuresw

acp = sd.PCA()
acp_features = acp.fit_transform(sp.scale(feat))

In [ ]:
res=OCS.predict(v)
for i in range(len(res)) : 
    if res[i]==1 : 
        res[i]=0
    else :
        res[i]= 1 
res=pd.DataFrame(res.T)
res.to_csv('IA.csv',sep = ';', mode = 'w',index=True,header=['seqID;anomaly'])
sum(res)